**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_numerical -p 8888:8888 -d palmoreck/jupyterlab_numerical:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_numerical
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

---

Nota generada a partir de [liga1](https://drive.google.com/file/d/1zCIHNAxe5Shc36Qo0XjehHgwrafKSJ_t/view), [liga2](https://drive.google.com/file/d/12L7rOCgW7NEKl_xJbIGZz05XXVrOaPBz/view).

In [61]:
!pip3 install --user -q cvxpy

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import os

In [2]:
cur_directory = os.getcwd()

In [3]:
dir_alg_python = '/algoritmos/Python'

In [4]:
os.chdir(cur_directory + dir_alg_python)

In [5]:
import math

import numpy as np

from numerical_differentiation import gradient_approximation, \
                                      Hessian_approximation
from line_search import line_search_for_residual_by_backtracking
from utils import compute_error


In [6]:
def log_barrier_aux_eval_function(eval_f_const_inequality):
    '''
    Auxiliary function for evaluation of contraint inequalities in logarithmic barrier
    '''
    #get values that are nonnegative through indexes
    idx_zeros = np.logical_and(eval_f_const_inequality < np.nextafter(0,1),
                               eval_f_const_inequality > -np.nextafter(0,1))
    idx_positive = eval_f_const_inequality > 0 
    idx  = np.logical_or(idx_zeros, idx_positive) 
    #eval constraint inequality functions
    #next line produces warning if a value of constraint is nonnegative
    eval_f_const_inequality = np.log(-eval_f_const_inequality)                                                           
    #assign large value for values positive or equal to 0
    eval_f_const_inequality[idx] = 1e10     
    return eval_f_const_inequality

In [7]:
def constraint_inequalities_funcs_eval(x):
    const_ineq_funcs_eval = np.array([const(x) for const in constraint_inequalities_funcs_generator()])
    return const_ineq_funcs_eval

In [8]:
def logarithmic_barrier(f,x, t_path):
    '''
    Implementation of Logarithmic barrier function.
    '''
    return t_path*f(x) + np.sum(log_barrier_aux_eval_function(constraint_inequalities_funcs_eval(x)))
           

In [9]:
def numerical_differentiation_of_logarithmic_barrier(f, x, t_path):
    '''
    First and second derivative of logarithmic barrier function approximation
    via finite differences
    '''
    sum_gf_const = 0
    sum_Hf_const = 0
    for const in constraint_inequalities_funcs_generator():
        const_eval = const(x)
        gf_const_eval = gradient_approximation(const, x)
        Hf_const_eval = Hessian_approximation(const, x)
        sum_gf_const += gf_const_eval/const_eval  
        sum_Hf_const += np.outer(gf_const_eval,gf_const_eval)/const_eval**2 - Hf_const_eval/const_eval
    gf_eval = gradient_approximation(f,x)
    Hf_eval = Hessian_approximation(f,x)
    return {'gradient': t_path*gf_eval-sum_gf_const,
            'Hessian': t_path*Hf_eval + sum_Hf_const
           }

In [10]:
def constraint_inequalities_funcs_generator():
    '''
    Generator for functional form of inequalities.
    For every example this function produces different fuctions.
    '''
    for k, v in const.items():
        yield v

# Example

In [11]:
from utils import compute_error

In [12]:
def line_search_for_log_barrier_by_backtracking(f,dir_desc,x,t_path,
                                                der_direct, alpha=.15, beta=.5):
    '''
    Line search that sufficiently decreases f restricted to a ray in the direction dir_desc.
    Args:
        alpha (float): parameter in line search with backtracking, tipically .15
        beta (float): parameter in line search with backtracking, tipically .5
        f (lambda expression): definition of function f.
        dir_desc (array): descent direction.
        x (array): numpy array that holds values where line search will be performed.
        der_direct (float): directional derivative of f.
    Returns:
        t (float): positive number for stepsize along dir_desc that sufficiently decreases f.
    '''
    t=1
    if alpha > 1/2:
        print('alpha must be less than or equal to 1/2')
        t=-1
    if beta>1:
        print('beta must be less than 1')
        t=-1;   
    if t!=-1:
        eval1 = logarithmic_barrier(f,x + t*dir_desc, t_path)
        eval2 = logarithmic_barrier(f,x, t_path) + alpha*t*der_direct
        while eval1 > eval2:
            t=beta*t
            eval1=logarithmic_barrier(f,x + t*dir_desc, t_path)
            eval2=logarithmic_barrier(f,x, t_path) + alpha*t*der_direct
    return t

In [13]:
def Newtons_method_log_barrier_feasible_init_point(f, A, constraints_ineq, 
                                                   t_path, x_0, tol, 
                                                   tol_backtracking, x_ast=None, p_ast=None, maxiter=30,
                                                   gf_symbolic = None,
                                                   Hf_symbolic = None):
    '''
    Newton's method to numerically approximate solution of min f subject to Ax = b.
    IMPORTANT: this implementation requires that initial point x_0, satisfies: Ax_0 = b
    Args:
        f (fun): definition of function f as lambda expression or function definition.
        A (numpy ndarray): 2d numpy array of shape (m,n) defines system of constraints Ax=b.
        
        constraints_ineq
        
        t_path
        
        x_0 (numpy ndarray): initial point for Newton's method. Must satisfy: Ax_0 = b
        tol (float): tolerance that will halt method. Controls stopping criteria.
        tol_backtracking (float): tolerance that will halt method. Controls value of line search by backtracking.
        x_ast (numpy ndarray): solution of min f, now it's required that user knows the solution...
        p_ast (float): value of f(x_ast), now it's required that user knows the solution...
        maxiter (int): maximum number of iterations
        gf_symbolic (fun): definition of gradient of f. If given, no approximation is
                                     performed via finite differences.
        Hf_symbolic (fun): definition of Hessian of f. If given, no approximation is
                                     performed via finite differences.
    Returns:
        x (numpy ndarray): numpy array, approximation of x_ast.
        iteration (int): number of iterations.
        Err_plot (numpy ndarray): numpy array of absolute error between p_ast and f(x) with x approximation
                          of x_ast. Useful for plotting.
        x_plot (numpy ndarray): numpy array that containts in columns vector of approximations. Last column
                        contains x, approximation of solution. Useful for plotting.
    '''
    iteration = 0
        
    x = x_0
    
    feval = f(x)
        
    if gf_symbolic and Hf_symbolic:
        gfeval = gf_symbolic(x)
        Hfeval = Hf_symbolic(x)
    else:
        grad_Hess_log_barrier_dict = numerical_differentiation_of_logarithmic_barrier(f,x,t_path)
        gfeval = grad_Hess_log_barrier_dict['gradient']
        Hfeval = grad_Hess_log_barrier_dict['Hessian']

    
    normgf = np.linalg.norm(gfeval)
    condHf= np.linalg.cond(Hfeval)
    
    Err_plot_aux = np.zeros(maxiter)
    Err_plot_aux[iteration]=compute_error(p_ast,feval)
    
    Err = compute_error(x_ast,x)
    
    if (np.all(A < np.nextafter(0,1))): #A is zero matrix
        system_matrix = Hfeval
        rhs = -gfeval
        n = x.size
        p = 0
        
    else:
        if(A.ndim == 1):
            p = 1
            n = x.size
            zero_matrix = np.zeros(p)
            first_stack = np.column_stack((Hfeval, A.T))
            second_stack = np.row_stack((A.reshape(1,n).T,zero_matrix)).reshape(1,n+1)[0]
        else:
            p,n = A.shape
            zero_matrix = np.zeros((p,p))
            first_stack = np.column_stack((Hfeval, A.T))
            second_stack = np.column_stack((A,zero_matrix))
            
        system_matrix = np.row_stack((first_stack,second_stack))
        zero_vector = np.zeros(p)
        rhs = -np.row_stack((gfeval.reshape(n,1), zero_vector.reshape(p,1))).T[0]
    
    x_plot = np.zeros((n,maxiter))
    x_plot[:,iteration] = x
    

    #Newton's direction and Newton's decrement
    dir_desc = np.linalg.solve(system_matrix, rhs)
    dir_Newton = dir_desc[0:n]
    dec_Newton = -gfeval.dot(dir_Newton)
    w_dual_variable_estimation = dir_desc[n:(n+p)]


    print('I\tNorm gfLogBarrier \tNewton Decrement\tError x_ast\tError p_ast\tline search\tCondHf')
    print('{}\t\t{:0.2e}\t\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{}\t\t{:0.2e}'.format(iteration,normgf,
                                                                         dec_Newton,Err,
                                                                         Err_plot_aux[iteration],"---",
                                                                         condHf))
    
    stopping_criteria = dec_Newton/2
    iteration+=1
    while(stopping_criteria>tol and iteration < maxiter):
        der_direct = -dec_Newton
        t = line_search_for_log_barrier_by_backtracking(f,dir_Newton,x,t_path,der_direct)
        x = x + t*dir_Newton
        feval = f(x)
            
        if gf_symbolic and Hf_symbolic:
            gfeval = gf_symbolic(x)
            Hfeval = Hf_symbolic(x)
        else:
            grad_Hess_log_barrier_dict = numerical_differentiation_of_logarithmic_barrier(f,x,t_path)
            gfeval = grad_Hess_log_barrier_dict['gradient']
            Hfeval = grad_Hess_log_barrier_dict['Hessian']
        
        if (np.all(A < np.nextafter(0,1))): #A is zero matrix
            system_matrix = Hfeval
            rhs = -gfeval
            n = x.size
            p = 0
        else:
            if(A.ndim == 1):
                first_stack = np.column_stack((Hfeval, A.T))
            else:
                first_stack = np.column_stack((Hfeval, A.T))
            system_matrix = np.row_stack((first_stack,second_stack))
            rhs = -np.row_stack((gfeval.reshape(n,1), zero_vector.reshape(p,1))).T[0]

        #Newton's direction and Newton's decrement
        dir_desc = np.linalg.solve(system_matrix, rhs)
        dir_Newton = dir_desc[0:n]
        dec_Newton = -gfeval.dot(dir_Newton)
        w_dual_variable_estimation = dir_desc[n:(n+p)]
        
        Err_plot_aux[iteration]=compute_error(p_ast,feval)
        x_plot[:,iteration] = x
        Err = compute_error(x_ast,x)
        print('{}\t\t{:0.2e}\t\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{:0.2e}'.format(iteration,normgf,
                                                                                dec_Newton,Err,
                                                                                Err_plot_aux[iteration],t,
                                                                                condHf))
        stopping_criteria = dec_Newton/2
        if t<tol_backtracking: #if t is less than tol_backtracking then we need to check the reason
            iter_salida=iteration
            iteration = maxiter - 1
        iteration+=1
    print('{} {:0.2e}'.format("Error of x with respect to x_ast:",Err))
    print('{} {}'.format("Approximate solution:", x))
    cond = Err_plot_aux > np.finfo(float).eps*10**(-2)
    Err_plot = Err_plot_aux[cond]
    
    if iteration == maxiter and t < tol_backtracking:
        print("Backtracking value less than tol_backtracking, check approximation")
        iteration=iter_salida
    else:
        if iteration == maxiter:
            print("Reached maximum of iterations, check approximation")
    x_plot = x_plot[:,:iteration]
    return [x,iteration,Err_plot,x_plot]

In [14]:
def Path_following_method(f, A, constraints_ineq,
                          x_0, tol,
                          tol_backtracking, x_ast=None, p_ast=None, maxiter=30,
                          mu=None,
                          gf_symbolic=None, Hf_symbolic=None,
                          tol_outer_iter=1e-6,maxiter_path=30
                          ):
    m = len(const.keys())
    if p_ast:
        t_0 = m/math.fabs(f(x_0)-p_ast) #other option: t_0 = mu*m/(f(x_0)-p_ast)
    else:
        print("p_ast must be defined")
    t = t_0
    x = x_0
    log_barrier_eval = logarithmic_barrier(f,x,t)
    stopping_criteria = m/t
    outer_iter = 1
    const_ineq_funcs_eval = constraint_inequalities_funcs_eval(x)
        
    print("Outer iterations of path following method")
    print('{} {:0.2e}'.format("Mu value:", mu))
    print('Outer iteration\tLogBarrier \tt_log_barrier\tStopping criteria')
    print('{}\t\t{:0.2e}\t{:0.2e}\t{:0.2e}'.format(outer_iter,log_barrier_eval,
                                                 t,stopping_criteria
                                                 ))
    print("----------------------------------------------------------------------------------------")
    iter_barrier = 0
    while(stopping_criteria > tol_outer_iter and outer_iter < maxiter_path):
        [x,iteration,Err_plot,x_plot] = Newtons_method_log_barrier_feasible_init_point(f, A, constraints_ineq, 
                                                                                       t, x_0, tol, 
                                                                                       tol_backtracking, x_ast, p_ast, maxiter,
                                                                                       gf_symbolic, Hf_symbolic)
        print("Inner iterations")
        print(x)
        const_ineq_funcs_eval = constraint_inequalities_funcs_eval(x)
        
        if(sum(const_ineq_funcs_eval > np.nextafter(0,1)) >=1):
            print("Some constraint inequalities evaluated in x were nonnegative, check approximations")
        
        t=mu*t
        log_barrier_eval = logarithmic_barrier(f,x,t)
        outer_iter+=1
        stopping_criteria = m/t
        print("----------------------------------------------------------------------------------------")
        print("Outer iterations of path following method")
        print('{} {:0.2e}'.format("Mu value:", mu))
        print('Outer iteration\tLogBarrier \tt_log_barrier\tStopping criteria')
        print('{}\t\t{:0.2e}\t{:0.2e}\t{:0.2e}'.format(outer_iter,log_barrier_eval,
                                                     t,stopping_criteria
                                                     ))
        print("----------------------------------------------------------------------------------------")
        iter_barrier+= iter_barrier + iteration
            
    return [x,iter_barrier,t]

# Primer ejemplo

$$ \min \quad x_1^2 + x_2^2 + x_3^2 + x_4^2 -2x_1-3x_4$$

$$\text{sujeto a: } $$

$$
\begin{array}{c}
2x_1 + x_2 + x_3 + 4x_4 = 7 \\
x_1 + x_2 + 2x_3 + x_4 = 6
\end{array}
$$

$$x_1, x_2, x_3, x_4 \geq 0$$

In [15]:
fo = lambda x: x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2-2*x[0]-3*x[3]

In [16]:
const = {0: lambda x: -x[0],
         1: lambda x: -x[1],
         2: lambda x: -x[2],
         3: lambda x: -x[3]
        }

In [17]:
A= np.array([[2,1,1,4],
             [1,1,2,1]])

In [18]:
b=np.array([7,6])

In [19]:
x_ast=np.array([1.1232876712328763,0.6506849315068493,
                1.8287671232876714,0.5684931506849317])

In [20]:
x_0 = np.array([8.082191780821915e-01,
                8.767123287671235e-01,
                1.821917808219178e+00,
                6.712328767123281e-01])

In [21]:
x_0

array([0.80821918, 0.87671233, 1.82191781, 0.67123288])

In [22]:
p_ast=fo(x_ast)

In [23]:
p_ast

1.4006849315068495

In [24]:
tol_outer_iter = 1e-6
tol=1e-8
tol_backtracking=1e-12
maxiter=30
mu=10

In [25]:
[x,iter_barrier,t] = Path_following_method(fo, A, const,
                                           x_0, tol,
                                           tol_backtracking, x_ast, p_ast, maxiter,
                                           mu, tol_outer_iter = tol_outer_iter 
                                          )

Outer iterations of path following method
Mu value: 1.00e+01
Outer iteration	LogBarrier 	t_log_barrier	Stopping criteria
1		3.87e+01	2.49e+01	1.61e-01
----------------------------------------------------------------------------------------
I	Norm gfLogBarrier 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0		1.09e+02		7.74e+00	1.73e-01	1.15e-01	---		1.04e+00
1		1.09e+02		3.10e-04	7.71e-03	2.27e-04	1.00e+00	1.04e+00
2		1.09e+02		3.10e-04	7.71e-03	2.27e-04	2.27e-13	1.04e+00
Error of x with respect to x_ast: 7.71e-03
Approximate solution: [1.11865825 0.66695832 1.8231155  0.56815242]
Backtracking value less than tol_backtracking, check approximation
Inner iterations
[1.11865825 0.66695832 1.8231155  0.56815242]
----------------------------------------------------------------------------------------
Outer iterations of path following method
Mu value: 1.00e+01
Outer iteration	LogBarrier 	t_log_barrier	Stopping criteria
2		3.48e+02	2.49e+02	1.61e-02
----------------------------

In [26]:
[x,iter_barrier,t]

[array([1.12328763, 0.65068512, 1.82876705, 0.56849314]),
 174,
 24851063.829786982]

In [27]:
compute_error(x_ast,x)

9.029491544987291e-08

# Comparación con [cvxpy](https://github.com/cvxgrp/cvxpy)

In [28]:
import cvxpy as cp

In [29]:
x1 = cp.Variable()
x2 = cp.Variable()
x3 = cp.Variable()
x4 = cp.Variable()


In [30]:
# Create two constraints.
constraints = [2*x1+x2+x3+4*x4-7 == 0,x1+x2+2*x3+x4-6 == 0,x1>=0,x2>=0,x3>=0,x4>=0]

# Form objective.

obj = cp.Minimize(x1**2+x2**2+x3**2+x4**2-2*x1-3*x4)

In [31]:
# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.

1.4006849315068515

In [32]:
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x1.value, x2.value, x3.value,x4.value)

status: optimal
optimal value 1.4006849315068515
optimal var 1.1232876712328763 0.6506849315068494 1.8287671232876717 0.5684931506849316


# Segundo ejemplo

$$\min 2x_1 + 5x_2$$

$$\text{sujeto a: }$$

$$
\begin{array}{c}
6-x_1-x_2 \leq 0 \\
-18 + x_1 +2x_2 \leq 0\\
x_1, x_2 \geq 0
\end{array}
$$

In [33]:
fo = lambda x: 2*x[0] + 5*x[1]

In [34]:
const = {0: lambda x: 6-x[0]-x[1],
         1: lambda x: -18+x[0]+2*x[1],
         2: lambda x: -x[0],
         3: lambda x: -x[1]
        }

In [35]:
A=np.array([0,0],dtype=float)
b = 0

In [36]:
x_ast = np.array([6,0], dtype=float)

In [37]:
x_0 = np.array([4,4], dtype=float)

In [38]:
p_ast=fo(x_ast)

In [39]:
p_ast

12.0

In [40]:
tol_outer_iter = 1e-3
tol=1e-8
tol_backtracking=1e-12
maxiter=30
mu=10

In [41]:
[x,iter_barrier,t] = Path_following_method(fo, A, const,
                                           x_0, tol,
                                           tol_backtracking, x_ast, p_ast, maxiter,
                                           mu, tol_outer_iter=tol_outer_iter
                                          )

Outer iterations of path following method
Mu value: 1.00e+01
Outer iteration	LogBarrier 	t_log_barrier	Stopping criteria
1		1.23e+01	2.50e-01	1.60e+01
----------------------------------------------------------------------------------------
I	Norm gfLogBarrier 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0		8.37e-01		5.59e+00	7.45e-01	1.33e+00	---		9.46e+00
1		8.37e-01		2.61e-02	2.15e-01	5.37e-01	5.00e-01	9.46e+00
2		8.37e-01		2.61e-02	2.15e-01	5.37e-01	1.78e-15	9.46e+00
Error of x with respect to x_ast: 2.15e-01
Approximate solution: [6.88318309 0.93672033]
Backtracking value less than tol_backtracking, check approximation
Inner iterations
[6.88318309 0.93672033]
----------------------------------------------------------------------------------------
Outer iterations of path following method
Mu value: 1.00e+01
Outer iteration	LogBarrier 	t_log_barrier	Stopping criteria
2		5.08e+01	2.50e+00	1.60e+00
------------------------------------------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in log
  if sys.path[0] == '':


In [42]:
[x,iter_barrier,t]

[array([6.00008070e+00, 7.00989697e-05]), 106, 25000.0]

In [43]:
compute_error(x_ast,x)

1.7815100934694886e-05

# Comparación con [cvxpy](https://github.com/cvxgrp/cvxpy)

In [44]:
x1 = cp.Variable()
x2 = cp.Variable()


In [45]:
# Create two constraints.
constraints = [6-x1-x2 <= 0,-18+x1+2*x2<=0,x1>=0,x2>=0]

# Form objective.

obj = cp.Minimize(2*x1+5*x2)



In [46]:
# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.


12.0000000016275

In [47]:
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x1.value, x2.value)

status: optimal
optimal value 12.0000000016275
optimal var 6.000000000175689 2.552244387851183e-10


**Referencias:**

* S. P. Boyd, L. Vandenberghe, Convex Optimization, Cambridge University Press, 2009.